# Agent using user logic function apps

In [109]:
# Imports
import json
import requests
from typing import Dict, Any, Callable

from azure.identity import DefaultAzureCredential
from azure.mgmt.logic import LogicManagementClient

In [110]:
class AzureLogicAppTool:
    """
    A service that manages multiple Logic Apps by retrieving and storing their callback URLs,
    and then invoking them with an appropriate payload.
    """

    def __init__(self, subscription_id: str, resource_group: str, credential=None):
        if credential is None:
            credential = DefaultAzureCredential()
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.logic_client = LogicManagementClient(credential, subscription_id)

        self.callback_urls: Dict[str, str] = {}

    def register_logic_app(self, logic_app_name: str, trigger_name: str) -> None:
        """
        Retrieves and stores a callback URL for a specific Logic App + trigger.
        Raises a ValueError if the callback URL is missing.
        """
        callback = self.logic_client.workflow_triggers.list_callback_url(
            resource_group_name=self.resource_group,
            workflow_name=logic_app_name,
            trigger_name=trigger_name,
        )

        if callback.value is None:
            raise ValueError(f"No callback URL returned for Logic App '{logic_app_name}'.")

        self.callback_urls[logic_app_name] = callback.value

    def invoke_logic_app(self, logic_app_name: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        """
        Invokes the registered Logic App (by name) with the given JSON payload.
        Returns a dictionary summarizing success/failure.
        """
        if logic_app_name not in self.callback_urls:
            raise ValueError(f"Logic App '{logic_app_name}' has not been registered.")

        url = self.callback_urls[logic_app_name]
        response = requests.post(url=url, json=payload)

        if response.ok:
            return {"result": f"Successfully invoked {logic_app_name}."}
        else:
            return {"error": (f"Error invoking {logic_app_name} " f"({response.status_code}): {response.text}")}

In [111]:
def create_get_user_accounts_function(service: AzureLogicAppTool, logic_app_name: str) -> Callable[[str, str, str], str]:
    """
    Returns a function that retrieve user accounts by invoking the specified Logic App in LogicAppService.
    This keeps the LogicAppService instance out of global scope by capturing it in a closure.
    """

    def get_user_accounts_via_logic_app(user_id: int) -> str:
        """
        Retrieves a list of accounts for the user.
        :param user_id: The ID of the user.
        :return: A JSON string with the list of accounts.
        :rtype: str
        """
        payload = {
            "user_id": user_id
        }
        result = service.invoke_logic_app(logic_app_name, payload)
        return json.dumps(result)

    return get_user_accounts_via_logic_app

In [112]:
import os
import requests
from typing import Set

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential

In [113]:
# Create the project client
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [114]:
# Extract subscription and resource group from the project scope
subscription_id = project_client.scope["subscription_id"]
resource_group = project_client.scope["resource_group_name"]

In [115]:
# Logic App details
logic_app_name = "get_user_accounts"
trigger_name = "When_a_HTTP_request_is_received"

In [116]:
# Create and initialize AzureLogicAppTool utility
logic_app_tool = AzureLogicAppTool(subscription_id, resource_group)
logic_app_tool.register_logic_app(logic_app_name, trigger_name)
print(f"Registered logic app '{logic_app_name}' with trigger '{trigger_name}'.")

Registered logic app 'get_user_accounts' with trigger 'When_a_HTTP_request_is_received'.


In [117]:
logic_app_tool.callback_urls

{'get_user_accounts': 'https://prod-06.eastus2.logic.azure.com:443/workflows/75a8695be7b649d7aa6fc22ef9696423/triggers/When_a_HTTP_request_is_received/paths/invoke?api-version=2019-05-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=BVOQhdXvB2mkAtp-R7gMvfFdeyivh7Fmu57rWK4PXjQ'}

In [118]:
# Create the specialized "send_email_via_logic_app" function for your agent tools
get_user_accounts_func = create_get_user_accounts_function(logic_app_tool, logic_app_name)

In [119]:
# Prepare the function tools for the agent
functions_to_use: Set = {
    get_user_accounts_func,  # This references the AzureLogicAppTool instance via closure
}

In [120]:
with project_client:
    # Create an agent
    functions = FunctionTool(functions=functions_to_use)
    toolset = ToolSet()
    toolset.add(functions)

    # To enable tool calls executed automatically
    project_client.agents.enable_auto_function_calls(toolset=toolset)

    # Create agent with explicit instructions
    agent_instructions = """
    You are a specialized agent for retrieving user account information.
    
    IMPORTANT: You have access to the following function:
    - get_user_accounts_via_logic_app(user_id: int) -> str
      This function takes a user ID as an integer and returns account information as a JSON string.
    
    When asked about user accounts:
    1. Extract the user ID from the request
    2. Call get_user_accounts_via_logic_app with the user ID parameter
    3. Return the account information in a readable format listing the accounts.
    4. If the user ID is not provided, respond with an error message.
    
    """

    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="Get user accounts agent",
        instructions=agent_instructions,
        toolset=toolset,
        
    )
    

    print(f"Created agent, ID: {agent.id}")
    print(f"Agent name: {agent.name}")
    print(f"Agent tools: {agent.tools}")
    print(f"Agent model: {agent.model}")

    #project_client.agents.enable_auto_function_calls(toolset=toolset)
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create a message in the thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Hello, please get the user accounts for user ID 1.",
    )

    print(f"Created message, ID: {message.id}")

    # Create and process an agent run in the thread
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

Created agent, ID: asst_HqTxGhnz1IrNlWcaNTiUhtdN
Agent name: Get user accounts agent
Agent tools: [{'type': 'function', 'function': {'name': 'get_user_accounts_via_logic_app', 'description': 'Retrieves a list of accounts for the user.', 'parameters': {'type': 'object', 'properties': {'user_id': {'type': 'integer', 'description': 'The ID of the user.'}}, 'required': ['user_id']}, 'strict': False}}]
Agent model: gpt-4o-mini
Created agent, ID: asst_HqTxGhnz1IrNlWcaNTiUhtdN
Created thread, ID: thread_Ys2y8VrRwF7dWLTMvHuw9Dw1
Created message, ID: msg_1qd1jEwh0mfgDPqQbbTYw7eQ
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_lqv8Wf4P2rla4Nn06uK4ONrV', 'object': 'thread.message', 'created_at': 1745195018, 'assistant_id': 'asst_HqTxGhnz1IrNlWcaNTiUhtdN', 'thread_id': 'thread_Ys2y8VrRwF7dWLTMvHuw9Dw1', 'run_id': 'run_vwV3zo7lfzNRZv4lUUnbjwkk', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The user accounts for us